In [ ]:
!pip install wandb --upgrade
!pip install transformers
!pip install wget
!pip install tqdm

In [ ]:
!wget https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/train.jsonl
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/val.jsonl
!wget  https://raw.githubusercontent.com/AI-Front/NTI/main/semifinals/data/test.jsonl

In [ ]:
from tqdm.notebook import tqdm
import re
import pandas as pd
import json
from transformers import AutoModel
import wandb
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def text_splitter(text, amount=500): 
    tokens = text.split(' ')
    new_text = ' '.join(tokens[-amount:])
    return new_text
    
def get_data(data_json_file):
    qw = 0
    df = pd.DataFrame(columns = ['Text', 'label'])
    with open(data_json_file, 'r') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in tqdm(json_list):
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    #data = {'Text': text_splitter(text+' Query: '+query+' Answer: '+a['text']), 'label': a['label']}
                    #data = {'Text' : 'Query: '+query+' Answer: '+a['text'] + ' Text: ' + text, 'label': a['label']}
                    data = {'Text' : query+' '+a['text'] + ' ' + text, 'label': a['label']}
                    #data = {'Text' : query+' '+a['text'], 'label': a['label']}
                    #data = {'Text' : text + ' Query: ' + query + ' Answer: ' + a['text'], 'label': a['label']}
                    #X.append(text+' Query: '+query+' Answer: '+a['text'])
                    #y.append(a['label'])
                    df.loc[qw] = data
                    qw += 1
    return df

In [ ]:
def simbols(frame):
    for i in tqdm(range(len(frame.index))):
        alpha = frame.iloc[i]['Text']
        #print(alpha)
        alpha = str(re.sub(r'\([^()]*\)', '', alpha))
        #alpha = str(alpha.replace('"', ''))
        alpha = str(alpha.replace('Ё', 'Е'))
        alpha = str(alpha.replace('ё', 'е'))
        
        frame.at[i, 'Text'] = alpha
    return frame

In [ ]:
train = get_data('train.jsonl')
val = get_data('val.jsonl')
train_data = simbols(train)
val_data = simbols(val)

In [ ]:
train_texts = train_data.Text.values
train_labels = train_data.label.values
val_texts = val_data.Text.values
val_labels = val_data.label.values
train_labels
train_labels = train_labels.astype('int')
val_labels = val_labels.astype('int')

Это не меняется

In [ ]:
import numpy as np

# Функция для вычисления точности модели
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    #Получаем время в секундах и возвращаем в формате hh:mm:ss  
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
  
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import torch  
	  
# Проверка на доступность видеокарты  
if torch.cuda.is_available():  
	  
	  # Объявляем в качестве устройства GPU  
	  device = torch.device("cuda")  
	  
	  print('Количество доступных GPU %d .' % torch.cuda.device_count())  
  
	  print('Используемая GPU:', torch.cuda.get_device_name(0))  
	  
# Если GPU недоступна  
else:  
	  print('GPU недоступна, используется CPU.')  
	  device = torch.device("cpu")  

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [ ]:
sweep_config = {
    #'method': 'grid', #grid, random
    "method": "bayes",  # grid, random
    "metric": {"name": "Validation_Loss", "goal": "minimize"},
    'parameters': {
        'epochs': {
            'values': [2, 3, 4]
        },
        'batch_size': {
            'values': [12]
        },
        
        'eps': {
            'values' : [1e-6, 1e-7, 1e-8]
        },

        'learning_rate': {
            "min": 1e-5, "max": 1e-4
        },
        'max_length': {
            'values': [512]
        }

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Sputnik")

Create sweep with ID: wimhyfn9
Sweep URL: https://wandb.ai/anonumous/Poisk3/sweeps/wimhyfn9


Сама модель

In [ ]:
from transformers import BertTokenizer
# Загрузим токенайзер для BERT
print('Загрузка токенайзера для BERT...')
tokenizer_m = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

Загрузка токенайзера для BERT...


In [ ]:
# Используем токенайзер для всего текста
import torch
def complite_data_one(sentenses, labels, max_length):
      input_ids = []
      attention_masks = []

      # Проходимся по предложениям
      for sent in tqdm(sentenses):
        
        encoded_dict = tokenizer_m.encode_plus(
                          sent,                      # Исходное предложение
                          add_special_tokens = True, # Добавляем токены 
                          max_length = max_length,           # Устанавливаем максимальную длинну
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Создаём марицу внимания
                          return_tensors = 'pt',     # Возвращаем тензоры
                   )
    
           
        input_ids.append(encoded_dict['input_ids'])
    
        
        attention_masks.append(encoded_dict['attention_mask'])

      # Конвертируем списки в тензоры
      input_ids = torch.cat(input_ids, dim=0)
      attention_masks = torch.cat(attention_masks, dim=0)
      #print(*labels)
      #labels = np.array(labels)
      labels = labels.astype('int')
      print(labels.dtype)
      labels = torch.from_numpy(labels)


      return input_ids, attention_masks, labels

Собираем всё воединно

In [ ]:
import random


def train():
    config_defaults = { # задаём конфиг модели по умолчанию
        'epochs': 2,
        'batch_size': 4,
        'learning_rate': 1e-3,
        'max_length': 512
    }
    wandb.init(config=config_defaults) # Объявляем wandb для автоподбора параметров
    config = wandb.config
    input_ids_train_m, attention_masks_train_m, labels_train_m = complite_data_one(train_texts, train_labels, # преобразуем наши дастасеты для обучения
                                                                                   config.max_length)
    input_ids_val_m, attention_masks_val_m, labels_val_m = complite_data_one(val_texts, val_labels, config.max_length) # повторяем тоже для валидации

    train_dataset_m = TensorDataset(input_ids_train_m, attention_masks_train_m, labels_train_m) # преобразуем в специальный тип данных
    val_dataset_m = TensorDataset(input_ids_val_m, attention_masks_val_m, labels_val_m)

    train_dataloader_m = DataLoader( # загружаем данные для обучения
        train_dataset_m,
        sampler=RandomSampler(train_dataset_m),
        batch_size=config.batch_size
    )

    validation_dataloader_m = DataLoader( # делаем ещё раз для валидации
        val_dataset_m,
        sampler=SequentialSampler(val_dataset_m),
        batch_size=config.batch_size
    )

    model_m = BertForSequenceClassification.from_pretrained(  # BertForSequenceClassification
        "bert-base-multilingual-cased",  # Используем 12 слойный мультиязычный BERT
        num_labels=2, # 2 метки для бинарной классификации

        output_attentions=False,
        output_hidden_states=False,
    )

    model_m.to(device) # объявляем, что модель будет работать на GPU

    optimizer_m = AdamW(model_m.parameters(),
                        lr=config.learning_rate,  # задаём learning rate
                        eps=config.eps  # задаём adam epsilon
                        )

    epochs = config.epochs #задаём колличество эпох

    total_steps_m = len(train_dataloader_m) * epochs

    scheduler_m = get_linear_schedule_with_warmup(optimizer_m,
                                                  num_warmup_steps=0,
                                                  num_training_steps=total_steps_m)

    seed_val = 796 # выбираем случайный сид

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # Статистика по тренировке
    training_stats = []

    # Уитываем время тренировки
    total_t0 = time.time()



    for epoch_i in range(0, epochs):

        print("")
        print('======== Эпоха {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Тренировка...')

        # Начальное время тренировки
        t0 = time.time()


        total_train_loss = 0

        # Переведём модель в режим тренировки
        model_m.train()


        for step, batch in enumerate(train_dataloader_m):

            # Будем обновлять данные каждые 40 шагов
            if step % 40 == 0 and not step == 0:
                # Ввычисляем время
                elapsed = format_time(time.time() - t0)

                # Печатаем прогресс.
                print('  Батч {:>5,}  из  {:>5,}.    Прошло: {:}.'.format(step, len(train_dataloader_m), elapsed))


            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            #Обнуляем модель
            model_m.zero_grad()

            # Получаем вывод из модели
            output = model_m(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

            loss = output.loss
            logits = output.logits
            # Вычисляем loss для тренировки
            total_train_loss += loss.item()


            loss.backward()

            # Вычисляем градиент
            torch.nn.utils.clip_grad_norm_(model_m.parameters(), 1.0)


            optimizer_m.step()

            # Обновляем learning rate
            scheduler_m.step()
            wandb.log({"batch loss": float(loss.item())}) # Добавляем в wandb значения

        # Вычисляем общий loss для тренировки
        avg_train_loss = total_train_loss / len(train_dataloader_m)

        # вычисляем время на эпоху
        training_time = format_time(time.time() - t0)
        wandb.log({"loss": float(avg_train_loss)})
        print("")
        print("  Средняя потеря на тренировке: {0:.2f}".format(avg_train_loss))
        print("  Время тренировки эпохи: {:}".format(training_time))



        print("")
        print("Запуск валидации...")

        t0 = time.time()

        # Сменяем режим модели
        model_m.eval()


        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0


        for batch in tqdm(validation_dataloader_m):

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)


            with torch.no_grad():
                # Получаем вывод модели для валидации
                output = model_m(b_input_ids,
                                 token_type_ids=None,
                                 attention_mask=b_input_mask,
                                 labels=b_labels)

                loss = output.loss
                logits = output.logits
            # Вычисляем потерю для валидации
            total_eval_loss += loss.item()


            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Вычисляем точность.
            total_eval_accuracy += flat_accuracy(logits, label_ids)
            wandb.log({"batch accuracy": float(total_eval_accuracy)})

        # Вычисляем финальную точность
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader_m)
        print("  Точность: {0:.4f}".format(avg_val_accuracy))
        wandb.log({"Accuracy": float(avg_val_accuracy)})

        avg_val_loss = total_eval_loss / len(validation_dataloader_m)

        # Высчитываем время валидации
        validation_time = format_time(time.time() - t0)
        wandb.log({"Validation_Loss": float(avg_val_loss)})
        print("  Потеря на валидации: {0:.2f}".format(avg_val_loss))
        print("  Время валидации: {:}".format(validation_time))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Обучение завершено!")

    print("Общее время {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))

In [ ]:
wandb.agent(sweep_id, train)